In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}
url = "https://hvl.instructure.com/api/v1/accounts/1/users"

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


# Slette gamle brukarar
Først les eg inn dei som er mest aktuelle å slette i første omgang:


In [3]:
andre = pd.read_csv("andre.csv")
utan_sis = pd.read_csv("utan_sis.csv")


## Finne sist pålogging
No kjem det vanskeleg arbeidet: å finne "siste pålogging" for kvar av desse? 

Eg kan prøve med endepunktet `/api/v1/users/:user_id/logins`: 

In [4]:
user_id = andre.iloc[-1]['id']
print(user_id)
url = f"https://hvl.instructure.com/api/v1/users/{user_id}/logins"
resultat = requests.get(url, headers=headers)
resultat.json()

95471


[{'id': 95984,
  'user_id': 95471,
  'account_id': 1,
  'workflow_state': 'active',
  'unique_id': 'daa015@uib.no',
  'created_at': '2023-08-07T07:26:50Z',
  'sis_user_id': 'hvl230807-p1-akv',
  'integration_id': None,
  'authentication_provider_id': None,
  'declared_user_type': None}]

Den ga meg fint lite, i alle fall ikkje det som står i dokumentasjonen. 

Neste steg er tabellen `enrollments`; her får eg feltet `last_activity_at` som i følge dokumentasjonen er 

> The last activity time of the user for the enrollment. This time stamp updates when a user navigates through the course using the Course Navigation menu, replies to a discussion or submits an assignment or quiz, and when they access course files and pages (does not include mobile interactions or group participation).

Det eg får ut er siste gang ein brukar var inne i eit gitt emne. Så eg må sjå på *alle* slike oppføringar for ein brukar, og så plukke ut den yngste. NB! Eg har henta data med skriptet "les_CD2_tabell.ipynb", men eg har berre lagra `user_id` og `course_id`. Dokumentasjonen tilrår å sjekke `course_section_id` og `role_id` også, for ein brukar kan jo ha vore logga på som student i eit emne og litt seinare som tilsett (eller studentassistent) i emnet. Men eg trur risikoen for dette er liten (det er jo eksterne faglærarar og kursdeltakarar dette).

In [6]:
data = pd.read_csv('brukarar_last_activity_at.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302226 entries, 0 to 302225
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   value.user_id           302226 non-null  int64 
 1   value.course_id         302226 non-null  int64 
 2   value.last_activity_at  223576 non-null  object
dtypes: int64(2), object(1)
memory usage: 6.9+ MB


In [7]:
utan_sis

,id,name,sis_user_id,created_at
0,1206,Katrine Abelsen,NaN,2017-11-06T13:36:18+01:00
1,1207,Enok Abrahamsen,NaN,2017-11-06T13:36:38+01:00
2,27329,ACS,NaN,2018-10-15T15:45:01+02:00
3,102310000000076186,Sumaya Mohammed Adan,NaN,2021-07-27T03:07:42+02:00
4,12788,Najma ahmed,NaN,2018-04-23T22:08:08+02:00
...,...,...,...,...
580,1288,Katrine (stud1) Aasekjær,NaN,2017-11-14T10:37:32+01:00
581,1289,Katrine (stud2) Aasekjær,NaN,2017-11-14T10:44:12+01:00
582,1290,Katrine (stud3) Aasekjær,NaN,2017-11-14T10:44:51+01:00
583,54983,Kariann Haaland Aasland,NaN,2020-12-03T13:58:45+01:00


Eg vil helst gå gjennom lista `utan_sis` og sjekke siste pålogging for alle der (basert på `id`). Alle som har blank eller tidlegare enn 2024 kan eg sikkert slette?

In [17]:
for id in utan_sis['id']:
    if len(data[data['value.user_id']==id])>0:
        print(f"{id}: {data[data['value.user_id']==id]}")

812:         value.user_id  value.course_id value.last_activity_at
125927            812              576                    NaN
96676:         value.user_id  value.course_id    value.last_activity_at
289170          96676            30265  2025-02-19T07:57:51.077Z
816:         value.user_id  value.course_id value.last_activity_at
125919            816              576                    NaN
684:         value.user_id  value.course_id value.last_activity_at
125473            684              575                    NaN
1506:        value.user_id  value.course_id value.last_activity_at
63923           1506              575                    NaN
829:         value.user_id  value.course_id value.last_activity_at
125999            829              576                    NaN
833:         value.user_id  value.course_id value.last_activity_at
126005            833              576                    NaN
837:         value.user_id  value.course_id value.last_activity_at
125989            837  